In [14]:
import pandas as pd

In [15]:


# Charger les fichiers CSV
operations = pd.read_csv("../data/operations.csv")
resultats_humain = pd.read_csv("../data/resultats_humain.csv")
flotteurs = pd.read_csv("../data/flotteurs.csv")

# Afficher les premières lignes
print("=== OPERATIONS ===")
print(operations.head(), "\n")

print("=== RESULTATS HUMAIN ===")
print(resultats_humain.head(), "\n")

print("=== FLOTTEURS ===")
print(flotteurs.head(), "\n")

# Vérifier les infos sur les colonnes et types
print("=== INFO OPERATIONS ===")
print(operations.info(), "\n")

print("=== INFO RESULTATS HUMAIN ===")
print(resultats_humain.info(), "\n")

print("=== INFO FLOTTEURS ===")
print(flotteurs.info(), "\n")


/tmp/ipykernel_9499/303084641.py:2: DtypeWarning: Columns (1,2,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  operations = pd.read_csv("../data/operations.csv")


=== OPERATIONS ===
   operation_id type_operation pourquoi_alerte              moyen_alerte  \
0       -135614            MAS             NaN                VHF phonie   
1       -135613            SAR             NaN  Téléphone mobile à terre   
2       -135598            MAS             NaN            Téléphone fixe   
3       -135597            MAS             NaN                VHF phonie   
4       -135596            SAR             NaN        Balise de détresse   

                           qui_alerte                  categorie_qui_alerte  \
0                     Navire impliqué                       Navire à la mer   
1        CORG / Gendarmerie nationale  Autorité militaire française à terre   
2  Autre organisme ou personne privée          Organisme ou personne privée   
3                     Navire impliqué                       Navire à la mer   
4                        CROSS / MRCC     Autorité civile française à terre   

                cross         departement  est_me

Pour chaque table, nous avons analysé :

    le nombre de lignes et colonnes,

    les types de données,

    les valeurs manquantes,

    les doublons,

    les valeurs aberrantes,

    la cohérence des dates,

    la cohérence de la clé operation_id.

Les résultats de cet audit ont permis de définir les règles de nettoyage (voir section suivante).

In [ ]:


def audit_table(df, table_name):
    print(f"\n================ AUDIT : {table_name} ================")

    # Dimensions
    print(f"Nombre de lignes : {df.shape[0]}")
    print(f"Nombre de colonnes : {df.shape[1]}")

    # Colonnes
    print("\nColonnes :")
    print(df.columns.tolist())

    # Types
    print("\nTypes des colonnes :")
    print(df.dtypes)

    # Valeurs manquantes
    print("\nValeurs manquantes :")
    print(df.isna().sum())

    # Doublons
    print("\nNombre de doublons :")
    print(df.duplicated().sum())

    # Statistiques descriptives
    print("\nStatistiques descriptives :")
    print(df.describe(include='all'))

    # Vérification des colonnes date/heure
    print("\nVérification des colonnes date/heure :")
    for col in df.columns:
        if "date" in col.lower() or "time" in col.lower():
            try:
                pd.to_datetime(df[col])
                print(f"{col} : format date valide")
            except:
                print(f" {col} : format date NON valide")

    print("=====================================================\n")


In [17]:
tables = {
    "operations": operations,
    "resultats_humain": resultats_humain,
    "flotteurs": flotteurs
}

for name, df in tables.items():
    audit_table(df, name)



================ AUDIT : operations ================
Nombre de lignes : 385782
Nombre de colonnes : 26

Colonnes :
['operation_id', 'type_operation', 'pourquoi_alerte', 'moyen_alerte', 'qui_alerte', 'categorie_qui_alerte', 'cross', 'departement', 'est_metropolitain', 'evenement', 'categorie_evenement', 'autorite', 'seconde_autorite', 'zone_responsabilite', 'latitude', 'longitude', 'vent_direction', 'vent_direction_categorie', 'vent_force', 'mer_force', 'date_heure_reception_alerte', 'date_heure_fin_operation', 'numero_sitrep', 'cross_sitrep', 'fuseau_horaire', 'systeme_source']

Types des colonnes :
operation_id                     int64
type_operation                  object
pourquoi_alerte                 object
moyen_alerte                    object
qui_alerte                      object
categorie_qui_alerte            object
cross                           object
departement                     object
est_metropolitain                 bool
evenement                       object
ca

In [18]:
print("=== Vérification des operation_id ===")

if "operation_id" in operations.columns:
    ops_ids = set(operations["operation_id"])
else:
    ops_ids = set()

for name, df in tables.items():
    if "operation_id" in df.columns:
        ids = set(df["operation_id"])
        print(f"\nTable {name} :")
        print(f"- Nombre d'IDs uniques : {len(ids)}")
        print(f"- IDs présents ici mais absents dans operations : {ids - ops_ids}")


=== Vérification des operation_id ===

Table operations :
- Nombre d'IDs uniques : 385782
- IDs présents ici mais absents dans operations : set()

Table resultats_humain :
- Nombre d'IDs uniques : 282559
- IDs présents ici mais absents dans operations : set()

Table flotteurs :
- Nombre d'IDs uniques : 309191
- IDs présents ici mais absents dans operations : set()


In [ ]:
print( "Résumé global ")
def resume_table(df, table_name):
    return {
        "table": table_name,
        "rows": df.shape[0],
        "cols": df.shape[1],
        "missing_values": df.isna().sum().sum(),
        "duplicates": df.duplicated().sum()
    }

resume_global = [resume_table(df, name) for name, df in tables.items()]
pd.DataFrame(resume_global)


,table,rows,cols,missing_values,duplicates
0,operations,385782,26,1470102,0
1,resultats_humain,290544,5,0,0
2,flotteurs,315191,7,280052,189


In [20]:
expected_columns = {
    "operations": ["operation_id", "date", "type", "lieu"],
    "resultats_humain": ["operation_id", "nb_sauves", "nb_deces"],
    "flotteurs": ["operation_id", "flotteur_id", "etat"]
}

for name, df in tables.items():
    print(f"\n=== Vérification colonnes attendues : {name} ===")
    missing = set(expected_columns[name]) - set(df.columns)
    extra = set(df.columns) - set(expected_columns[name])
    print("Colonnes manquantes :", missing)
    print("Colonnes en trop :", extra)



=== Vérification colonnes attendues : operations ===
Colonnes manquantes : {'date', 'lieu', 'type'}
Colonnes en trop : {'zone_responsabilite', 'evenement', 'longitude', 'date_heure_reception_alerte', 'categorie_qui_alerte', 'seconde_autorite', 'fuseau_horaire', 'cross', 'qui_alerte', 'numero_sitrep', 'departement', 'mer_force', 'latitude', 'vent_direction_categorie', 'type_operation', 'vent_direction', 'moyen_alerte', 'date_heure_fin_operation', 'systeme_source', 'pourquoi_alerte', 'categorie_evenement', 'autorite', 'vent_force', 'est_metropolitain', 'cross_sitrep'}

=== Vérification colonnes attendues : resultats_humain ===
Colonnes manquantes : {'nb_sauves', 'nb_deces'}
Colonnes en trop : {'categorie_personne', 'nombre', 'dont_nombre_blesse', 'resultat_humain'}

=== Vérification colonnes attendues : flotteurs ===
Colonnes manquantes : {'etat', 'flotteur_id'}
Colonnes en trop : {'pavillon', 'numero_ordre', 'categorie_flotteur', 'resultat_flotteur', 'type_flotteur', 'numero_immatricul

In [21]:
print("\n=== Vérification avancée des operation_id ===")

if "operation_id" in operations.columns:
    ops_unique = operations["operation_id"].nunique()
    ops_dups = operations["operation_id"].duplicated().sum()
    print(f"IDs uniques dans operations : {ops_unique}")
    print(f"Doublons dans operations : {ops_dups}")

for name, df in tables.items():
    if "operation_id" in df.columns:
        orphan = set(df["operation_id"]) - set(operations["operation_id"])
        print(f"\nTable {name} : {len(orphan)} IDs orphelins")
        if len(orphan) > 0:
            print(orphan)



=== Vérification avancée des operation_id ===
IDs uniques dans operations : 385782
Doublons dans operations : 0

Table operations : 0 IDs orphelins

Table resultats_humain : 0 IDs orphelins

Table flotteurs : 0 IDs orphelins


Vérification de la clé commune operation_id

In [22]:
for name, df in tables.items():
    if "operation_id" in df.columns:
        print(f"{name} : valeurs uniques operation_id = {df['operation_id'].nunique()}")


operations : valeurs uniques operation_id = 385782
resultats_humain : valeurs uniques operation_id = 282559
flotteurs : valeurs uniques operation_id = 309191


vérifier cohérence entre tables

Table OPERATIONS — Analyse
✔ Taille

    385 782 lignes

    26 colonnes

✔ Problèmes détectés

    Beaucoup de colonnes object (texte) → normal.

    Colonnes avec beaucoup de valeurs manquantes :

        pourquoi_alerte → 176k manquants

        departement → 76k manquants

        seconde_autorite → 370k manquants (quasi vide)

        latitude / longitude → 54k manquants

        vent_direction, vent_force, mer_force → 100k manquants

➡️ Ces colonnes devront être traitées dans le nettoyage.

✔ Dates

    date_heure_reception_alerte

    date_heure_fin_operation

Elles sont en string, mais le format semble valide (2025-06-11 22:50:00+00:00).

➡️ À convertir en datetime.

✔ Valeurs aberrantes possibles

    operation_id est négatif → est-ce normal ?
    Probablement oui (identifiants internes CROSS).

Table OPERATIONS — Analyse
✔ Taille

    385 782 lignes

    26 colonnes

✔ Problèmes détectés

    Beaucoup de colonnes object (texte) → normal.

    Colonnes avec beaucoup de valeurs manquantes :

        pourquoi_alerte → 176k manquants

        departement → 76k manquants

        seconde_autorite → 370k manquants (quasi vide)

        latitude / longitude → 54k manquants

        vent_direction, vent_force, mer_force → 100k manquants

➡️ Ces colonnes devront être traitées dans le nettoyage.
✔ Dates

    date_heure_reception_alerte

    date_heure_fin_operation

Elles sont en string, mais le format semble valide (2025-06-11 22:50:00+00:00).

➡️ À convertir en datetime.
✔ Valeurs aberrantes possibles

    operation_id est négatif → est-ce normal ?
    Probablement oui (identifiants internes CROSS).

🟦 3. Table RESULTATS HUMAIN — Analyse
✔ Taille

    290 544 lignes

    5 colonnes

✔ Propreté

    Aucune valeur manquante

    Types cohérents

    operation_id toujours présent

➡️ Cette table est propre.
🟦 4. Table FLOTTEURS — Analyse
✔ Taille

    315 191 lignes

    7 colonnes

✔ Problèmes détectés

    numero_ordre → 70k valeurs manquantes

    numero_immatriculation → 180k manquants

    pavillon → 20k manquants

➡️ Colonnes à nettoyer ou à laisser en nullable.
🟦 5. Interprétation globale
✔ Les trois tables sont cohérentes

Elles ont toutes un operation_id.
✔ Beaucoup de valeurs manquantes dans OPERATIONS

C’est normal pour des données opérationnelles.
✔ Les dates sont en string

À convertir.
✔ Les types sont corrects sauf quelques colonnes mixtes

À nettoyer avant Pandera.


Voici une **interprétation complète et professionnelle** de tous les résultats que tu viens d’obtenir.  
Tu peux **copier-coller directement** ce bloc dans ton notebook (cellule Markdown).  
Il synthétise **tout l’audit**, et prépare parfaitement la section “Nettoyage”.

---

# 🧾 **Interprétation complète des résultats de l’audit**

L’audit des trois tables (`operations`, `resultats_humain`, `flotteurs`) met en évidence la structure, la qualité et la cohérence des données avant la phase de nettoyage.  
Les résultats permettent d’identifier les colonnes problématiques, les incohérences potentielles et les transformations nécessaires pour la suite du pipeline.

---

## 📌 **1. Table `operations`**

### **Structure générale**
- **385 782 lignes**, **26 colonnes**
- Colonnes majoritairement de type `object` (texte), quelques colonnes numériques (`float64`, `int64`) et une colonne booléenne (`est_metropolitain`)

### **Qualité des données**
- **1 470 102 valeurs manquantes** au total  
  → c’est la table la plus incomplète du dataset
- Colonnes très incomplètes :
  - `pourquoi_alerte`
  - `departement`
  - `seconde_autorite` (quasi vide)
  - `latitude`, `longitude`
  - `vent_direction`, `vent_force`, `mer_force`
- Colonnes de dates (`date_heure_reception_alerte`, `date_heure_fin_operation`) au format texte mais **format valide**
- **Aucun doublon** détecté
- **Aucun ID orphelin** → tous les `operation_id` existent dans cette table

### **Colonnes attendues vs colonnes réelles**
- Colonnes attendues manquantes : `date`, `lieu`, `type`  
  → Ces colonnes n’existent pas dans le dataset réel  
- Colonnes “en trop” : toutes les colonnes métier réelles  
  → Cela montre que le dictionnaire attendu n’est pas aligné avec les données fournies

### **Conclusion**
La table `operations` est riche mais hétérogène, avec beaucoup de valeurs manquantes et des colonnes complexes.  
Elle nécessitera un nettoyage approfondi, notamment sur :
- les types
- les dates
- les colonnes géographiques
- les colonnes météo
- les colonnes d’autorité

---

## 📌 **2. Table `resultats_humain`**

### **Structure générale**
- **290 544 lignes**, **5 colonnes**

### **Qualité des données**
- **0 valeur manquante**
- Types cohérents (`int64` et `object`)
- Aucun doublon
- Aucun ID orphelin

### **Colonnes attendues vs réelles**
- Colonnes attendues manquantes : `nb_sauves`, `nb_deces`
- Colonnes présentes : `categorie_personne`, `resultat_humain`, `nombre`, `dont_nombre_blesse`

➡️ Le dictionnaire attendu ne correspond pas aux données réelles.  
➡️ Il faudra adapter le schéma Pandera et le modèle SQL à la réalité des données.

### **Conclusion**
Table propre, exploitable immédiatement après harmonisation des noms de colonnes.

---

## 📌 **3. Table `flotteurs`**

### **Structure générale**
- **315 191 lignes**, **7 colonnes**

### **Qualité des données**
- **280 052 valeurs manquantes**
- 189 doublons détectés
- Colonnes incomplètes :
  - `numero_ordre`
  - `pavillon`
  - `numero_immatriculation`

### **Colonnes attendues vs réelles**
- Colonnes attendues manquantes : `etat`, `flotteur_id`
- Colonnes présentes : `pavillon`, `numero_ordre`, `categorie_flotteur`, etc.

➡️ Comme pour les autres tables, le dictionnaire attendu ne correspond pas aux données réelles.

### **Conclusion**
Table globalement exploitable, mais nécessitant :
- suppression ou traitement des doublons
- gestion des valeurs manquantes
- harmonisation des colonnes

---

## 📌 **4. Cohérence de la clé `operation_id`**

### **Résultats**
- `operations` : **385 782 IDs uniques**
- `resultats_humain` : **282 559 IDs uniques**
- `flotteurs` : **309 191 IDs uniques**
- **0 ID orphelin dans les deux tables secondaires**
- **0 doublon dans `operations`**

### **Conclusion**
`operation_id` est une **clé primaire fiable** dans `operations`  
et une **clé étrangère cohérente** dans les deux autres tables.

Cela valide la possibilité de :
- créer un schéma relationnel propre (PostgreSQL)
- faire des jointures fiables
- construire un modèle analytique robuste

---

## 📌 **5. Synthèse générale**

| Table | Lignes | Colonnes | Valeurs manquantes | Doublons | Qualité |
|-------|--------|----------|---------------------|----------|---------|
| operations | 385 782 | 26 | 1 470 102 | 0 | ⚠️ Très hétérogène |
| resultats_humain | 290 544 | 5 | 0 | 0 | ✔ Propre |
| flotteurs | 315 191 | 7 | 280 052 | 189 | ⚠️ Moyenne |

---

## 📌 **6. Implications pour le nettoyage**

Les résultats de l’audit montrent que les actions suivantes seront nécessaires :

### 🔧 **Nettoyage des types**
- Conversion des dates en `datetime`
- Harmonisation des colonnes mixtes (`object` + `float`)

### 🔧 **Gestion des valeurs manquantes**
- Décider des colonnes à conserver ou supprimer
- Imputation ou suppression selon pertinence métier

### 🔧 **Nettoyage des doublons**
- Suppression des 189 doublons dans `flotteurs`

### 🔧 **Harmonisation des colonnes**
- Adapter les schémas Pandera aux colonnes réelles
- Renommer certaines colonnes si nécessaire

### 🔧 **Préparation du schéma SQL**
- `operations` → table principale (PK = operation_id)
- `resultats_humain` → table secondaire (FK = operation_id)
- `flotteurs` → table secondaire (FK = operation_id)

---

Quand tu veux, tu peux m’envoyer :

- les résultats de `df.describe()`
- les colonnes avec le plus de valeurs manquantes
- les types exacts après conversion

